# Prueba 4

Vamos a tratar de hacer una red que aprenda la XOR

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, sampler


import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
class TrainDataset(Dataset):
    
    def __init__(self,n_samples):
        self.in_data = torch.randint(low=0,high=2,size=(n_samples,2), requires_grad=False)
        self.out_data = self.in_data[:,0] * self.in_data[:,1]

    def __len__(self):
        return len(self.out_data)
    
    def __getitem__(self,idx):
        return self.in_data[idx], self.out_data[idx]
    
class ValidationDataset(Dataset):
    
    def __init__(self):
        self.in_data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], requires_grad=False)
        self.out_data = torch.tensor([0, 0, 0, 1], requires_grad=False)

    def __len__(self):
        return len(self.in_data)
    
    def __getitem__(self,idx):
        return self.in_data[idx], self.out_data[idx]

batch_size = 64
NUM_TRAIN = 49000

train_dataset = TrainDataset(NUM_TRAIN)
train_dataloader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

val_dataset = ValidationDataset()
val_dataloader = DataLoader(val_dataset,
                            batch_size=4)

class OneLayerNet(nn.Module):
    
    def __init__(self):
        super(OneLayerNet, self).__init__()
        self.fc1 = nn.Linear(2,1)
        
    def forward(self, x):
        return self.fc1(x)
    
model = OneLayerNet()

In [40]:
epochs = 1
val_accuracy_hist = []
loss_hist = []
device = 'cpu'

model = model.to(device=device)
for e in range(epochs):
    for phase in ['train', 'val']:
        
        if phase == 'train':
            model.train()
            for t, (x,y) in enumerate(train_dataloader):
                x = x.to(device=device, dtype=torch.float32)
                y = y.to(device=device, dtype=torch.float32)
                score = model(x).view(-1)
                loss = F.binary_cross_entropy(F.sigmoid(score),y)
                
                loss.backward()
                
        elif phase == 'val':
            model.eval()
            for t, (x,y) in enumerate(val_dataloader):
                x = x.to(device=device, dtype=torch.float32)
                y = y.to(device=device, dtype=torch.float32)
                score = model(x).view(-1)
                loss = F.binary_cross_entropy(F.sigmoid(score),y)
                loss_hist.append(loss)
                print(score)
                print(pred)
                

tensor([-0.3916, -0.9593, -0.1542, -0.7219], grad_fn=<ViewBackward>)
tensor([0, 0, 0, 0])
